# Pump probe experiments

## Setting

In [ ]:
## Setting for Prodigy connection
import spd_controller.Specs.Prodigy as Prodigy
from spd_controller.Specs.convert import itx
prodigy = Prodigy.RemoteIn(host="127.0.0.1")
prodigy.connect()

In [ ]:
## Setting for Stepping motor controller
import spd_controller.sigma.sc104 as sc104
sc104 = sc104.SC104()

## 1. Single measurement with FAT mode

In [ ]:
# Configuration
prodigy.clear()  # <- clear the buffer.
prodigy.defineFAT(start_energy=5, end_energy=6, step=0.02, dwell=.3)
prodigy.checkFAT(start_energy=5, end_energy=6, step=0.02, dwell=.3)  # Not absolutely required.
prodigy.set_excitation_energy(5.039)  # <- Set the excitation photon energy
prodigy.validate()

In [ ]:
parameters = prodigy.param
parameters

In [ ]:
prodigy.start(). # <- strat to measure

In [ ]:
data = prodigy.get_data()

In [ ]:
prodigy.save_data("PES_30_1.itx", id=1, comment="", measure_mode="FAT") #  <- comment format is same as usual: "variable_name:value;"
# Don't use the existing file name. The file is overwrittien.

## 2. Multi-scan measurement with SFAT (snapshot) mode

Due to the prodigy side limitation, the number of scan must be unity.  To realize the multiple scanning, use scan command() instead of start().

In [ ]:
# Configuration
prodigy.clear().  # <- clear the buffer.
prodigy.defineSFAT(start_energy=5, end_energy=6, dwell=.3)
prodigy.checkSFAT(start_energy=5, end_energy=6, dwell=.3) # Not absolutely required.
prodigy.set_excitation_energy(5.039)  # <- Set the excitation photon energy
prodigy.validate()

In [ ]:
parameters = prodigy.param
print(parameters)

In [ ]:
prodigy.scan(num_scan=3)

In [ ]:
prodigy.save_data('PES_30_2.itx', id=2, comment="", measure_mode="SFAT")

## 3. Moving steping motor

In [ ]:
### General syntax
### sc104.move_to_origin() # <- Goto mechanica origin.
sc104.move_abs(pos=220) # <- Move the mirror stage to the input position.
sc104.move_rel(move=10, miron=True) # <- Move the mirror stage by the input distance from the current position.
                                    # if micron=True is set, the value moving length is micon unit.
sc104.position() # <- Returns the current position.

## 4. Pump-probe experiments

In [ ]:
### Configureing the pump-probe experiments condition.
stage_start = 220
end_stage = 225
stage_move = 100 # micron unit
spectrum_id = 1
file_name_prefix = "PES_30_pumpprobe"

### 

In [ ]:
# Configuration
prodigy.clear().  # <- clear the buffer.
prodigy.defineSFAT(start_energy=5, end_energy=6, dwell=.3)
prodigy.checkSFAT(start_energy=5, end_energy=6, dwell=.3) # Not absolutely required.
prodigy.set_excitation_energy(5.039)  # <- Set the excitation photon energy
prodigy.validate()

### Do the pump-probe measurement.

In [ ]:
current_position = sc104.position()
while current_position < end_stage:
    prodigy.scan(num_scan=3)
    prodigy.save_data('{}_{:04}.itx'.format(file_name_prefix,spectrum_id),
                      id=spectrum_id,
                      comment="position:{:.4f};".format(current_position), measure_mode="SFAT")
    sc104.move_rel(move=10, micron=True)
    current_position = sc104.position()
    spectrum_id +=1  # Increment the spectrum_id